# BLOCKCHAIN ASSIGNMENT GROUP 54 (PROOF OF STAKE)

Visweswar Sirish Parupudi (2020AAPS0330H),
Konkala Rithvik (2020A8PS0517H),
Gautham Gutta (2020AAPS2204H),
Gokul Pradeep (2018B5A70785H)





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing the packages required for the task

In [2]:
pip install merkletools==1.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install pysha3==1.0b1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing various extensions

In [4]:
from hashlib import sha256
from merkletools import MerkleTools
import json
import time
import datetime
import random
import numpy as np

/usr/local/lib/python3.7/dist-packages/merkletools/__init__.py:7: UserWarning: sha3 is not working!
  warn("sha3 is not working!")


This function is a part of merkletools and the sha 256 is the algorithm we use hence passed in it.

In [5]:
mt = MerkleTools(hash_type="sha256")     
# This function is a part of merkletools and the sha 256 is the algorithm we use hence passed in it.


# Creating a Block

Defining the Block class which has different properties as specified below.

In [6]:
class Block: 
    def __init__(self, index, transactions, timestamp, previous_hash,root):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.root = root

    def display_block(self): #this is to store the block in the form of a dictionary to be visible later on the chain
        block={'index' : self.index,
              'transactions' : self.transactions,
              'prev hash' : self.previous_hash,
              'root' : self.root,
              'hash' : self.hash,
              'timestamp' : self.timestamp}
        return block

    def compute_hash(self): #this function computes the hash of our block
        block_string = json.dumps(self.__dict__, sort_keys=True)
        k= sha256(block_string.encode()).hexdigest()
        self.hash = k
        return k

# Creating our Blockchain

In [7]:
chaina=[]
chainb=[] #this is our blockchain in which our block objects get stored

Defining the Class Blockchain with properties and functions as shown below

In [8]:
class Blockchain:
  
    def __init__(self):
        
        self.unconfirmed_transactions = [] #pool of transactions to be added to a block

    def create_genesis_block(self): #function to create genesis block

        genesis_block = Block(0, [], 0, "0","0") #calling Block class to create an object for genesis block with required parameters
        genesis_block.hash = genesis_block.compute_hash() #computing hash of genesis block
        chaina.append(genesis_block)
        g_block = genesis_block.display_block()
        chainb.append(g_block) #adding the genesis block onto our chain
    
    def compute_root(self): #funtion to calculate merkle root of all transactions on our block

      for i in range(len(self.unconfirmed_transactions)): 
        mt.add_leaf(str(self.unconfirmed_transactions[i]), True) #we make use of merkletools library for this
      
      leaf_count =  mt.get_leaf_count(); #we get a count of leaf nodes i.e transactions
      mt.make_tree(); #the merkle tree is formed
      is_ready = mt.is_ready 
      root_value = mt.get_merkle_root(); #we compute the value of root hash
      mt.reset_tree() #the tree is reset
      return root_value

    def last_block(self): #function to return latest block in the chain
      return chaina[-1]

    def add_blocka(self, block):
        previous_hash = self.last_block().hash
        chaina.append(block)
        return True

    def add_blockb(self, block): #function to add a block into the chain
        previous_hash = self.last_block().hash #linking the prev block hash to current block
        chainb.append(block)
        return True

    def transactions(self,buyer,seller,property_,amount): #function to store transactions in a dictionary with required features
      self.transaction = {'buyer' : buyer,
                        'seller' : seller,
                        'property' : property_,
                        'amount' : amount,
                        'timestamp' : str(datetime.datetime.now())}

    def add_new_transaction(self): #to add a new transaction to our pool of transactions
        self.unconfirmed_transactions.append(self.transaction) 

    def mine(self): #this is the function required to mine a block and add it to the chain 

        if not self.unconfirmed_transactions:
            return False

        latest_block = self.last_block()
        
        new_block = Block( #adding a new block with following features
            index=latest_block.index + 1, #index of block
            transactions=self.unconfirmed_transactions, #transactions on the block
            timestamp=time.ctime(), #timestamp when the block is created
            previous_hash=latest_block.hash, #previous hash of the block
            root=self.compute_root() #merkle root of the block
        )

        new_block.compute_hash()
        n_block= new_block.display_block()

        self.add_blocka(new_block)
        self.add_blockb(n_block)#adding our block to the chain

        return True

In [9]:
moneyRangeMin = 32   #we are defining a random range of money/stake to be used in the pos algorithm
moneyRangeMax = 100000
disableTime = 50 #no. of iterations node can't forge after being chosen

# Participants child class of Blockchain class

Defining the Participant class which has properties like name, address etc

In [10]:
class Participant(Blockchain): #this is a child class of blockchain to register participants with required features
    def __init__(self,name,property_,idNumber,amount):
        super().__init__() #inheriting the parent class's init function
        self.name = name
        self.properties = property_
        self.idNumber = idNumber
        self.amount = amount
        self.stackedCoin = random.randint(moneyRangeMin,moneyRangeMax) #generating a random initial amount as stake
        self.time = 1
        self.coin_age = self.stackedCoin #Initialize with stackedCoin because time = 1
        self.selected = 0
        self.desactivated = 0
       
    def display_node(self): #function to display/store participant as a dictionary
        node={'name' : self.name,
              'amount' : self.amount,
              'id' : self.idNumber}
        return node

    def reset(self): # When a participant is chosen, the time is reset
        self.time = 1
        self.coin_age = self.time * self.stackedCoin

    def unable(self):
        self.desactivated = disableTime #Number of iteration a Participant is unable to forge after being chosen

    def timePass(self): # called at each iterration to reduce the time a participant is disable
        if self.desactivated > 0:
            self.desactivated -= 1

    def select(self): #Number of time a participant is selected
        self.selected += 1

    # def removeMoney(self,sub): #Remove coins
    #     self.stackedCoin = self.stackedCoin - sub
    #     self.coin_age = self.time * self.stackedCoin #refresh coin_age

    # def addMoney(self,add): #Add coins
    #     self.stackedCoin = self.stackedCoin + add
    #     self.coin_age = self.time * self.stackedCoin #refresh coin_age

    def increaseTime(self):
        self.time += 1
        self.coin_age = self.time * self.stackedCoin # stackedCoin * time, the time he didnt create a block, reset when he is chosen
            

# PROOF OF STAKE CONSENSUS ALGORITHM

In [11]:
def minerSelection(participants): # Take all the coin_age to sum it and do a weighted selection
#this is our consensus algorithm where one person is chosen randomly after they put up a stake which increases the chance for beinf selected based on coin age
      totalCoinAge = sum([participant.coin_age for participant in participants if participant.desactivated == 0])
      probabilities = (participant.coin_age/totalCoinAge for participant in participants if participant.desactivated == 0)
      chosenOne = random.choices([participant for participant in participants if participant.desactivated == 0], weights=probabilities, k=1)[0] #Return a participant depending on a probability given by the coin_age
      chosenOne.reset() #Reset his timer
      chosenOne.unable()
      return chosenOne


def pos(chosen):
          chosen.select()
          chosenPerson.append(chosen)
          chosen.mine() # The chosen one verify and add the block to the blockchain

          for participant in participants: #Increase timer of other participants
              participant.timePass() # Reduce timer of being disable for each participant having desactivated > 0
              if participant.idNumber != chosen.idNumber:
                  participant.increaseTime()


In [12]:
chosenPerson = [] #list of people chosen to mint/forge the block

# Registring new users with property

This is the algorithm for Inputting participants to make transactions and mine the blocks.Participants can be added as seen fit and when the participant list is done the algorithm moves onto the blocks.

In [13]:
participants= []#list of participants
names_list=[]
while(True): #we run a loop to register participants into the blockchain 
  addP = input('press 1 to add participants, press 0 to finalize the participant list ')
  l=0
  if(addP=='1'):
    print("Enter the details of paticipant")
    name= input('name:')
    names_list.append(name)
    property_=[item for item in input("Enter properties : ").split()]
    idno=input('id no:')
    amount=input('funds available for the person:')
    participants.append(Participant(str(name),property_,str(idno),str(amount))) #creating participant objects based on inputs, and appending them to out list of participants
    print("The current no of participants is ",len(participants))
  
  if(addP=='0' and l != 0):
    break
  if(addP=='0' and l==0):
    print("No participants were added please add participants")
    break

press 1 to add participants, press 0 to finalize the participant list 1
Enter the details of paticipant
name:sirish
Enter properties : vyas vk
id no:0330
funds available for the person:1000
The current no of participants is  1
press 1 to add participants, press 0 to finalize the participant list 1
Enter the details of paticipant
name:rithvik
Enter properties : shankar vm
id no:0514
funds available for the person:500
The current no of participants is  2
press 1 to add participants, press 0 to finalize the participant list 1
Enter the details of paticipant
name:gautham
Enter properties : ram budh
id no:2204
funds available for the person:600
The current no of participants is  3
press 1 to add participants, press 0 to finalize the participant list 0
No participants were added please add participants


# IMPLEMENTING OUR BLOCKCHAIN

Algorithm for mining blockchain we can add transactions and specify when the block can be mined.

Here we used a block with max of 6 transactions as a demo the number can be changed very easily. (In this case an option of  block being mined with lesser number of transactions is also enabled )

The algorithm asks for buyer and seller input and the amount of transaction which it then proceeds to add on block

In [14]:
a=Blockchain() #initialising our blockchain
a.create_genesis_block() #creating our genesis block
transNo = 0
while(True): #we start our loop to take in transactions
  transNo = 0
  print("Add details of trasnactions to be added onto the block")
  print("A maximum of 6 transactions can be added onto a block")
  miner=minerSelection(participants) #we select our miner based on the Proof Of Stake algorithm as implemented above

  while(True):
      transNo += 1;
      print("This is transaction number ",transNo," on this block\n")
      buyer = input('Buyer = ') #we start taking inputs
      seller = input('Seller = ')
      property1=[]
      property1 = input('property = ')
      amount = input('amount = ')
      
      buyerV = 0 
      sellerV = 0 
      buyerAV = 0
      buyerI = 0
      sellerI = 0
      propertyI = 0

      for i in range(len(participants)):  #we are checking if our seller is a valid user registered to the system
        if (seller == participants[i].name): #this is to make sure the seller is only selling property that they own and not fraudulent property
          for k in range(len(participants[i].properties)):
            if participants[i].properties[k]==property1:
              print("valid seller")
              propertyI = k
              sellerV = 1
              sellerI = i

      for j in range(len(participants)):  #we are checking if our buyer is a valid user registered to the system
        if (buyer == participants[j].name):
          print("valid buyer") 
          buyerV = 1
          buyerI = j
          
      if (int(participants[buyerI].amount)> int(amount)): #we check if buyer can afford the transaction based on funds available in his account
          buyerAV = 1
          participants[buyerI].amount = int(participants[buyerI].amount)- int(amount) #we deduct the transaction amount from the buyer after the transaction
          participants[sellerI].amount= int(participants[sellerI].amount)+ int(amount) #we increase the amount in the seller's wallet
          participants[buyerI].properties.append(participants[sellerI].properties[propertyI]) #we transfer the property ownership from seller to buyer
          participants[sellerI].properties.remove(participants[sellerI].properties[propertyI]) #the seller no longer owns said property



      if(buyerV == 1 and sellerV == 1 and buyerAV == 1): #if all conditions satsify we add the transaction to our pool of transactions
        miner.transactions(buyer,seller,property1,amount)
        miner.add_new_transaction()

        if (transNo > 6): #the block will get automatically mined if transactions exceed 6
          print("There are already 6 transactions a block will be mined.")
          pos(miner) #the chosen miner from the pos algorithm mines and adds the block to the chain
          print("chosen", miner.name) #we can check who mined the block this time
          print(chainb)
          print("The next transactions will be added on a seperate block")
          transNo = 0
          break

        if (transNo != 0): #we can add transactions and mine the block at said time
          select = input('enter 1 to add more transactions, enter 0 to mine a block = ')
          if (select == '0'):
            pos(miner) #the chosen miner from the pos algorithm mines and adds the block to the chain
            print("chosen", miner.name)#we can check who mined the block this time
            print(chainb) #we can see the blockchain
            transNo = 0
            print("Block has been generated with the specified transactions")
            print("You can continue adding transactions")
            break

      if(buyerAV==0):
        print("Buyer has insufficient funds")
      
      if(buyerV==0):
        print("Invalid Buyer")

      if(sellerV==0):
        print("Invalid Seller")

      if(buyerV == 0 or sellerV == 0 or buyerAV == 0):
        print("Please enter valid details")
        transNo-=1;

Add details of trasnactions to be added onto the block
A maximum of 6 transactions can be added onto a block
This is transaction number  1  on this block

Buyer = sirish
Seller = rithvik
property = vm
amount = 200
valid seller
valid buyer
enter 1 to add more transactions, enter 0 to mine a block = 1
This is transaction number  2  on this block

Buyer = gautham
Seller = sirish
property = vyas
amount = 300
valid seller
valid buyer
enter 1 to add more transactions, enter 0 to mine a block = 1
This is transaction number  3  on this block

Buyer = rithvik
Seller = fraud
property = scam
amount = 100000
valid buyer
Buyer has insufficient funds
Invalid Seller
Please enter valid details
This is transaction number  3  on this block

Buyer = rithvik
Seller = sirish
property = vk
amount = 200
valid seller
valid buyer
enter 1 to add more transactions, enter 0 to mine a block = 1
This is transaction number  4  on this block

Buyer = gautham
Seller = sirish
property = ram
amount = 400
valid buyer
Buy

KeyboardInterrupt: ignored

In [15]:
prop=input('enter property')
for i in range(len(chainb)):
  for j in range(len(chainb[i]['transactions'])):
    if chainb[i]['transactions'][j]['property']==prop:
      print(chainb[i]['transactions'][j])

#this function is to check transaction history of a property, input the property that is to be checked

enter propertyvk
{'buyer': 'rithvik', 'seller': 'sirish', 'property': 'vk', 'amount': '200', 'timestamp': '2022-10-17 18:02:49.321392'}
{'buyer': 'gautham', 'seller': 'rithvik', 'property': 'vk', 'amount': '30', 'timestamp': '2022-10-17 18:03:34.099062'}


In [16]:
for i in range(len(participants)): #this shows the net worth and property owned by all participants
  print("The participant ",participants[i].name," has a net amount of",participants[i].amount, "and property", participants[i].properties)

The participant  sirish  has a net amount of 1250 and property ['vm', 'shankar']
The participant  rithvik  has a net amount of 580 and property []
The participant  gautham  has a net amount of 270 and property ['ram', 'budh', 'vyas', 'vk']
